In [ ]:
#Auto-reload modules (used to develop functions outside this notebook)
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import numpy as np
import pandas as pd
import labrotation.file_handling as fh
import datadoc_util as ddutil
from labrotation import two_photon_session as tps
import h5py
from datetime import datetime as dt
from datetime import timedelta
import matplotlib.pyplot as plt

In [ ]:
env_dict = dict()
if not os.path.exists("./.env"):
    print(".env does not exist")
else:
    with open("./.env", "r") as f:
        for line in f.readlines():
            l = line.rstrip().split("=")
            env_dict[l[0]] = l[1]
print(env_dict.keys())

In [ ]:
if "SERVER_SYMBOL" in env_dict.keys():
    SERVER_SYMBOL = env_dict["SERVER_SYMBOL"]
else:
    SERVER_SYMBOL = "R"
    print(f"Server symbol not found in .env file. Setting it to {SERVER_SYMBOL}")

In [ ]:
output_dir = fh.open_dir("Choose export directory for results!")

In [ ]:
ddoc = ddutil.DataDocumentation(env_dict["DATA_DOCU_FOLDER"])

In [ ]:
ddoc.loadDataDoc()

In [ ]:
ddoc.setDataDriveSymbol(SERVER_SYMBOL)

In [ ]:
ddoc.checkFileConsistency()

In [ ]:
ddoc.GROUPING_DF[ddoc.GROUPING_DF["uuid"] == "513ed3d7c54042edab0f64df10882446"]

In [ ]:
def get_datetime_for_fname():
    now = dt.now()
    return f"{now.year:04d}{now.month:02d}{now.day:02d}-{now.hour:02d}{now.minute:02d}{now.second:02d}"

In [ ]:
matlab_2p_folder = env_dict["MATLAB_2P_FOLDER"]

# Define recordings to be used

In [ ]:
df_groupings = ddoc.getRecordingsWithExperimentType(["chr2_ctl", "chr2_sz", "chr2_szsd", "chr2_sd", "chr2_lfpsz_sd", "jrgeco_sd", "jrgeco_ctl"])

In [ ]:
# do the filtering by mouse
dset_dict = {"all ChR2": ["T370", "T413", "T430", "T452", "OPI-2239", "WEZ-8917", "WEZ-8924", "WEZ-8922"],
            "filtered ChR2": ["OPI-2239", "WEZ-8917", "WEZ-8924", "WEZ-8922"],
            "filtered ChR2 CA1+NC": ["OPI-2239", "WEZ-8917", "WEZ-8924", "WEZ-8922", "T413", "T430"]}

df_groupings = df_groupings[df_groupings["mouse_id"].isin(dset_dict["filtered ChR2 CA1+NC"])]

In [ ]:
df_groupings.head()

In [ ]:
df_groupings.mouse_id.unique()

# Consistency check
If no output, then everything looks fine

In [ ]:
for i_grouping, grouping in df_groupings.iterrows():
    nd2_fpath = SERVER_SYMBOL + os.path.join(grouping["folder"], grouping["nd2"])[1:]  # replace server symbol
    lv_fpath = SERVER_SYMBOL + os.path.join(grouping["folder"], grouping["labview"])[1:] 
    lvtime_fpath = SERVER_SYMBOL + os.path.join(grouping["folder"], os.path.splitext(grouping["labview"])[0]+"time.txt")[1:] 
    nikmeta_fpath = SERVER_SYMBOL + os.path.join(grouping["folder"], grouping["nikon_meta"])[1:] 
    
    
    if type(grouping["lfp"]) is float:  # NaN has type float, otherwise should be string
        lfp_fpath = None
    else:
        lfp_fpath =  SERVER_SYMBOL + os.path.join(grouping["folder"], grouping["lfp"])[1:] 
        if not os.path.exists(lfp_fpath):
            print(lfp_fpath)
    if not os.path.exists(nd2_fpath):
        print(nd2_fpath)
    if not os.path.exists(lv_fpath):
        print(lv_fpath)
    if not os.path.exists(lvtime_fpath):
        print(lvtime_fpath)
    if not os.path.exists(nikmeta_fpath):
        print(nikmeta_fpath)

In [ ]:
append_to_file = False
if append_to_file:
    existing_uuids = []
    original_fpath = fh.open_file("Choose h5 file to append to!")
    with h5py.File(original_fpath, "r") as hf:
        for uuid in hf.keys():
            existing_uuids.append(uuid)

In [ ]:
sessions_dict = dict()
has_lfp_dict = dict()
for i_grouping, grouping in df_groupings.iterrows():
    uuid = grouping.uuid
    if append_to_file:
        if uuid in existing_uuids:
            print(f"skipping {uuid}...")
            continue
    if uuid in sessions_dict.keys():
        print(f"skipping {uuid}...")
        continue
    
    nd2_fpath = os.path.join(grouping["folder"], grouping["nd2"]) # replace server symbol
    lv_fpath =  os.path.join(grouping["folder"], grouping["labview"])
    lvtime_fpath =  os.path.join(grouping["folder"], os.path.splitext(grouping["labview"])[0]+"time.txt") 
    nikmeta_fpath =  os.path.join(grouping["folder"], grouping["nikon_meta"])
    
    has_lfp = True
   
    
    if type(grouping["lfp"]) is float:  # NaN has type float, otherwise should be string
        lfp_fpath = None
        has_lfp = False
    else:
        lfp_fpath =  SERVER_SYMBOL + os.path.join(grouping["folder"], grouping["lfp"])[1:] 
    
    session = tps.TwoPhotonSession.init_and_process(nd2_fpath, nikmeta_fpath, lv_fpath, lvtime_fpath, lfp_fpath, matlab_2p_folder)
    
    sessions_dict[uuid] = session
    has_lfp_dict[uuid] = has_lfp

In [ ]:
if not append_to_file:
    output_assembled_fpath = os.path.join(output_dir, f"assembled_traces_{get_datetime_for_fname()}_ChR2.h5")
    print(f"Saving traces to {output_assembled_fpath}")

In [ ]:
def create_dt(t):
    t1 = t[1:]
    t0 = t[:-1]
    dt = np.zeros(len(t))
    dt[1:] = t1 - t0
    dt[0] = dt[1]  # assume same step size to avoid 0
    return dt
def create_totdist_abs(speed, dt):
    totdist_abs = np.zeros(len(speed))
    totdist_abs[0] = speed[0]*dt[0]
    for i in range(1, len(totdist_abs)):
        totdist_abs[i] = totdist_abs[i-1] + abs(speed[i]*dt[i])
    return totdist_abs

In [ ]:
if append_to_file:
    export_fpath = original_fpath
    access_type = "r+"
else:
    access_type = "w-"
    export_fpath = output_assembled_fpath

In [ ]:
with h5py.File(export_fpath, access_type) as hf:
    for event_uuid in sessions_dict.keys():
        
        if append_to_file:
            if event_uuid in existing_uuids:
                continue
        
        session = sessions_dict[event_uuid]
        exp_type = ddoc.getExperimentTypeForUuid(event_uuid) 
        
        segments = ddoc.getSegmentsForUUID(event_uuid).sort_values(by="frame_begin")
        if len(segments[segments["interval_type"] == "stimulation"]) > 0:
            stim_segment = segments[segments["interval_type"] == "stimulation"].iloc[0]
        else:
            print(f"Skipping recording with no data doc entry: {event_uuid}")
            continue
            #raise Exception()
        n_frames = len(sessions_dict[event_uuid].mean_fluo)
        
        # bl frames: beginning until stimulation
        # am frames: from first SD wave frame until end (to be consistent with TMEV am definition)
        #            if no SD: then from end of stimulation
        n_bl_frames = stim_segment["frame_begin"] - 1  # frames 1 to x = x frames. stim_segment["frame_begin"] is x+1 
        n_stim_frames = stim_segment["frame_end"] - stim_segment["frame_begin"] + 1 # both end points inclusive -> +1 
        i_stim_begin_frame = stim_segment["frame_begin"] - 1
        i_stim_end_frame = stim_segment["frame_end"] - 1
        
        
        df = ddoc.getSegmentsForUUID(uuid)
        df = df[df["interval_type"] == "stimulation"].iloc[0]
        i_stim_begin = df.frame_begin - 1
        i_stim_end = df.frame_end - 1
        
        
        
        if "sd_wave" in segments.interval_type.unique():
            # first frame of SD wave until end
            n_am_frames = n_frames - segments[segments["interval_type"] == "sd_wave"].iloc[0].frame_begin + 1
        else:
            # first frame AFTER end of stim until end
            n_am_frames = n_frames - segments[segments["interval_type"] == "stimulation"].iloc[0].frame_end
        event_uuid_grp = hf.create_group(event_uuid)
        #df_attributes = df_events[df_events["event_uuid"] == event_uuid]
        
        mouse_id = ddoc.getMouseIdForUuid(event_uuid)
        
        event_uuid_grp.attrs["session_uuids"] = [event_uuid]  # only one uuid, as ChR2 protocol is always single recording
        event_uuid_grp.attrs["has_lfp"] = [has_lfp_dict[event_uuid]]
        event_uuid_grp.attrs["window_type"] =  ddoc.getMouseWinInjInfo(mouse_id).window_type.iloc[0]
        event_uuid_grp.attrs["n_frames"] = n_frames
        event_uuid_grp.attrs["mouse_id"] = mouse_id
        event_uuid_grp.attrs["exp_type"] = exp_type
        event_uuid_grp.attrs["i_stim_begin_frame"] = i_stim_begin_frame
        event_uuid_grp.attrs["i_stim_end_frame"] = i_stim_end_frame
        # add stim time
        event_uuid_grp.attrs["stim_duration_s"] = df_groupings[df_groupings["uuid"] == event_uuid].iloc[0].stim_length
        

        event_uuid_grp.attrs["n_bl_frames"] = n_bl_frames
        event_uuid_grp.attrs["n_am_frames"] = n_am_frames
        event_uuid_grp.attrs["i_stim_begin_frame"] = i_stim_begin
        event_uuid_grp.attrs["i_stim_end_frame"] = i_stim_end
        
        lv_dist = session.belt_scn_dict['distance']
        lv_speed = session.belt_scn_dict['speed']
        lv_running = session.belt_scn_dict['running']
        lv_totdist = session.belt_scn_dict['totdist']
        lv_rounds = session.belt_scn_dict['rounds']
        lv_t_s = session.belt_scn_dict['tsscn']/1000.
        mean_fluo = session.mean_fluo
        
        lv_dt = create_dt(lv_t_s)
        lv_totdist_abs = create_totdist_abs(lv_speed, lv_dt)
        


        # get lfp data
        # lfp already matched to labview. lfp and movement channels t values should be same, but save them to be sure

        if has_lfp_dict[event_uuid]:
            lfp_t, lfp_y = session.lfp_lfp()
            lfp_mov_t, lfp_mov_y = session.lfp_movement()
        else:
            lfp_t = lv_t_s.copy()
            lfp_mov_t = lv_t_s.copy()

            lfp_y = np.zeros(len(lfp_t))
            lfp_mov_y = np.zeros(len(lfp_t))
                
        event_uuid_grp.attrs["n_lfp_steps"] = len(lfp_t)
        event_uuid_grp.attrs["n_lfp_mov_steps"] = len(lfp_mov_t)
        
        event_uuid_grp.create_dataset("lfp_mov_t", data=lfp_mov_t)
        event_uuid_grp.create_dataset("lfp_mov_y", data=lfp_mov_y)
        event_uuid_grp.create_dataset("lfp_t", data=lfp_t)
        event_uuid_grp.create_dataset("lfp_y", data=lfp_y)
        event_uuid_grp.create_dataset("lv_dist", data=lv_dist)
        event_uuid_grp.create_dataset("lv_dt", data=lv_dt)
        event_uuid_grp.create_dataset("lv_rounds", data=lv_rounds)
        event_uuid_grp.create_dataset("lv_running", data=lv_running)
        event_uuid_grp.create_dataset("lv_speed", data=lv_speed)
        event_uuid_grp.create_dataset("lv_t_s", data=lv_t_s)
        event_uuid_grp.create_dataset("lv_totdist", data=lv_totdist)
        event_uuid_grp.create_dataset("lv_totdist_abs", data=lv_totdist_abs)
        event_uuid_grp.create_dataset("mean_fluo", data=mean_fluo)
        
        
        # replicate joint_session_metadata_dict from tmev assembled traces
        if "sd_wave" in segments.interval_type.unique():
            break_points = np.array([0, n_bl_frames, n_bl_frames + n_stim_frames, segments[segments["interval_type"] == "sd_wave"].iloc[0].frame_begin + 1])
        else:
            break_points = np.array([0, n_bl_frames, n_bl_frames + n_stim_frames, segments[segments["interval_type"] == "stimulation"].iloc[0].frame_end + 1])
        
        event_uuid_grp.attrs["break_points"] = break_points
        event_uuid_grp.attrs["break_points_lfp"] = np.searchsorted(lfp_t, lv_t_s[break_points])
        
        segment_type_break_points = np.array([row["frame_begin"] - 1 for i_row, row in ddoc.getSegmentsForUUID(event_uuid).sort_values(by="frame_begin").iterrows()])
        event_uuid_grp.attrs["segment_type_break_points"] = segment_type_break_points
        event_uuid_grp.attrs["segment_type_break_points_lfp"] = np.searchsorted(lfp_t, lv_t_s[segment_type_break_points])
        event_uuid_grp.attrs["recording_break_points"] =  np.array([0])
        event_uuid_grp.attrs["recording_break_points_lfp"] = np.array([0]) 
        
        event_uuid_grp.attrs["recording_break_points_lfp"] = np.array([0]) 
        
        
        if "Time [s]" in session.df_stim.columns:
            event_uuid_grp.attrs["stim_start_end_time"] = np.array(session.df_stim["Time [s]"])
        else:
            event_uuid_grp.attrs["stim_start_end_time"] = np.array(session.df_stim["Time [m:s.ms]"])
            
        event_uuid_grp.attrs["stim_start_end_sw_time"] = np.array(session.df_stim["SW Time [s]"])  
        event_uuid_grp.attrs["stim_start_end_nidaq_time"] = np.array(session.df_stim["NIDAQ Time [s]"])  
    
    
        # get first imaging frame falling into stim, last imaging frame falling into stim
print(f"Saved to {export_fpath}")


In [ ]:
export_fpath

# Quality control
TODO: do a waterfall plot of both locomotion traces to check matching quality

## Load data

In [ ]:
mean_fluos = []
lv_speeds = []
lv_ts = []
lfp = []
lfp_speeds = []
lfp_ts = []
has_lfp_arr = []
uuids_lis = []

with h5py.File(export_fpath, "r") as hf:
    for event_uuid in hf.keys():
        uuids_lis.append(event_uuid)
        has_lfp = hf[event_uuid].attrs["has_lfp"]
        #print(has_lfp)
        #print(f'{len(hf[event_uuid]["lfp_mov_t"][()])} {len(hf[event_uuid]["lv_t_s"][()])}')
        mean_fluos.append(hf[event_uuid]["mean_fluo"][()])
        lv_speeds.append(hf[event_uuid]["lv_speed"][()])
        lv_ts.append(hf[event_uuid]["lv_t_s"][()])
        lfp.append(hf[event_uuid]["lfp_y"][()])
        lfp_speeds.append(hf[event_uuid]["lfp_mov_y"][()])
        lfp_ts.append(hf[event_uuid]["lfp_mov_t"][()])
        lfp_mov_t_data = hf[event_uuid]["lfp_mov_t"][()]
        has_lfp_arr.append(hf[event_uuid].attrs["has_lfp"])

## Normalize traces

In [ ]:
ddoc.getSegmentsForUUID(uuids_lis[0])

In [ ]:
amplitude = 100
mean_fluos_norm = []
lfp_norm = []
lv_speeds_norm = []
lfp_speeds_norm = []

def norm_trace(trace):
    min_trace = np.min(trace)
    max_trace = np.max(trace)
    return (trace - min_trace)/(max_trace - min_trace)

for i_event in range(len(lv_speeds)):
    print(i_event)
    df_event = ddoc.getSegmentsForUUID(uuids_lis[i_event])
    mean_fluo_raw = mean_fluos[i_event]
    i_begin_stim = df_event[df_event["interval_type"] == "stimulation"].frame_begin.iloc[0] - 1
    i_end_stim = df_event[df_event["interval_type"] == "stimulation"].frame_end.iloc[0]
    mean_fluo_raw[i_begin_stim:i_end_stim] = np.max(mean_fluo_raw[i_end_stim:])
    
    mean_fluo = amplitude*norm_trace(mean_fluo_raw)
    lfp_y = amplitude*norm_trace(lfp[i_event])
    lv_y = amplitude*norm_trace(lv_speeds[i_event])
    lfp_mov_y = amplitude*norm_trace(lfp_speeds[i_event])
    
    
    mean_fluos_norm.append(mean_fluo)
    lfp_norm.append(lfp_y)
    lv_speeds_norm.append(lv_y)
    lfp_speeds_norm.append(lfp_mov_y)
    

In [ ]:
offset = 0.0
fig = plt.figure(figsize=(18,72))
for i_event in range(len(lv_speeds)):
    color1="black"
    color2="grey"
    color3="blue"
    color4="green"
    lw = 1
    if not has_lfp_arr[i_event]:
        lw = 4
        color1="red"
        color2="orange"
    plt.plot(lv_ts[i_event], lv_speeds_norm[i_event]+offset, linewidth=lw, c=color1)
    plt.text(320, offset+0.2*amplitude, s=uuids_lis[i_event], fontdict={"fontsize":20, "color": "red"})
    offset += 0.5*amplitude
    plt.plot(lfp_ts[i_event], lfp_speeds_norm[i_event]+offset, linewidth=lw, c=color2)
    offset += amplitude
    plt.plot(lfp_ts[i_event], lfp_norm[i_event]+offset, linewidth=lw, c=color3)
    offset += amplitude
    plt.plot(lv_ts[i_event], mean_fluos_norm[i_event]+offset, linewidth=lw, c=color4)
    offset += 1.5*amplitude
    
ax = plt.gca()
plt.xlim((250, 400))
plt.tight_layout()
plt.savefig("C:\\Data\\lfpmismatch.pdf")
plt.show()

In [ ]:
# plot matched nikon, lfp, lv loco, plus open lv complete, uncut trace and adjust it. Goal is to see how much loco is cut out before

In [ ]:
l = []
lv_t = []
lv_speed = []
with open(os.path.normpath("R:\\AG-Wenzel\\Group\\jrgeco\\OPI_2239_jrgeco\\locomotion_exp\\231018_OPI_2239_optostim_d2\\OPI2239.181023.1541.txt"), "r") as f:
    for line in f.readlines():
        line_list = line.rstrip().split("\t")
        l.append(line_list)
        lv_t.append(int(line_list[8]))
        lv_speed.append(int(line_list[1]))

In [ ]:
lv_dict = {}
for uuid in uuids_lis:
    df_session = ddoc.getSessionFilesForUuid(uuid)
    lv_fpath = os.path.join(os.path.normpath(df_session["folder"].iloc[0]), df_session["labview"].iloc[0])
    l = []
    lv_t = []
    lv_speed = []
    with open(lv_fpath, "r") as f:
        for line in f.readlines():
            line_list = line.rstrip().split("\t")
            l.append(line_list)
            lv_t.append(int(line_list[8]))
            lv_speed.append(int(line_list[1]))
    if len(l[-1]) < len(l[0]):
        lv_t = lv_t[:-1]
        lv_speed = lv_speed[:-1]
    assert len(lv_speed) == len(lv_t)
    lv_dict[uuid] = [np.array(lv_t), np.array(lv_speed)]

In [ ]:
offset = 0.0
fig = plt.figure(figsize=(18,72))
for i_event in range(len(lv_speeds)):
    uuid = uuids_lis[i_event]
    color1="black"
    color2="red"
    color3="blue"
    color4="green"
    lw = 1
    if not has_lfp_arr[i_event]:
        lw = 4
        color1="red"
        color2="orange"
    plt.plot(lv_ts[i_event], lv_speeds_norm[i_event]+offset, linewidth=lw, c=color1)
    plt.text(320, offset+0.2*amplitude, s=uuid, fontdict={"fontsize":20, "color": "red"})
    offset += amplitude
    plt.plot(lv_dict[uuid][0]/1000., amplitude*norm_trace(lv_dict[uuid][1])+offset, linewidth=lw, c=color2)
    offset += amplitude
    plt.plot(lfp_ts[i_event], lfp_norm[i_event]+offset, linewidth=lw, c=color3)
    offset += amplitude
    plt.plot(lv_ts[i_event], mean_fluos_norm[i_event]+offset, linewidth=lw, c=color4)
    offset += 1.5*amplitude
    
ax = plt.gca()
plt.xlim((200, 500))
plt.tight_layout()
plt.savefig("C:\\Data\\lfp_vs_lv.pdf")
plt.show()

In [ ]:
# idea: take original raw data as well as tps original frequency but cut data. find latter in the former to get time stamps!

In [ ]:
ses = tps.TwoPhotonSession.init_and_process_uuid("09e0277c86234572ac586ab18be1cd58")

In [ ]:
# TODO: add uncut stuff to assembled_traces?!

In [ ]:
lv_t_uncut, lv_y_uncut = lv_dict["09e0277c86234572ac586ab18be1cd58"]

In [ ]:
lv_y_cut = ses.belt_dict["speed"]

In [ ]:
lv_t_cut = ses.belt_dict["time"]

In [ ]:
lv_t_uncut = lv_t_uncut/1000.

In [ ]:
lv_t_cut = lv_t_cut/1000.

In [ ]:
# TODO: find the time where the 250-400 seconds happens (in frame indices)!

In [ ]:
subset = lv_y_uncut[250:400]

In [ ]:
subset = lv_y_cut[np.logical_and(lv_t_cut > 250, lv_t_cut < 400) ]

In [ ]:
subset = lv_y_cut[5000:10000]

In [ ]:
lv_y_uncut

In [ ]:
lv_y_cut

In [ ]:
len(lv_y_uncut) - len(lv_y_cut)

In [ ]:
for i_begin in range(0, len(lv_y_cut) - len(subset)):
    if np.array_equal(subset, lv_y_uncut[i_begin:i_begin+len(subset)]):
        print(i_begin)
        break

In [ ]:
plt.plot(lv_y_uncut[5000:10000])

In [ ]:
for i_begin in range(len(lv_y_uncut) - len(lv_y_cut)):
    if np.array_equal(lv_y_cut, lv_y_uncut[i_begin:i_begin+2000]):
        print(i_begin)
        break

In [ ]:
plt.plot(lv_y_cut[:1000])

In [ ]:
plt.plot(lv_y_uncut[450:1550])

In [ ]:
subset = lv_y_cut[:1000]

In [ ]:
c_lis = []
for i_begin in range(len(lv_y_uncut) - len(subset)):
    corrcoef = np.corrcoef(subset, lv_y_uncut[i_begin:i_begin + len(subset)])[0, 1]
    if np.isnan(corrcoef):
        c_lis.append(0)
    else:
        c_lis.append(corrcoef)

In [ ]:
c_lis = np.array(c_lis)

In [ ]:
lv_t_uncut_shifted = lv_t_uncut - lv_t_uncut[425]

In [ ]:
fig = plt.figure(figsize=(18, 10))

plt.plot(lv_t_uncut_shifted, norm_trace(lv_y_uncut))
plt.plot(lv_t_cut, norm_trace(lv_y_cut))
plt.show()

In [ ]:
plt.plot(lv_y_uncut[425:425+len(subset)])

In [ ]:
plt.plot(c_lis)

In [ ]:
plt.plot(lv_y_cut[i_begin:i_begin+len(subset)])

In [ ]:
for i_event in range(len(uuids_lis)):
    uuid = uuids_lis[i_event]
    lv_t_uncut, lv_y_uncut = lv_dict[uuid]
    

# Manually review bad sessions

In [ ]:
i_bad = [11, 41, 63, 76]

In [ ]:
bad_sessions = dict()
for ind in i_bad:
    uuid = uuids_lis[ind]
    grouping = ddoc.getSessionFilesForUuid(uuid)
    grouping = grouping.iloc[0]
    
    
    nd2_fpath = os.path.join(grouping["folder"], grouping["nd2"])
    lv_fpath = os.path.join(grouping["folder"], grouping["labview"])
    lvtime_fpath = os.path.join(grouping["folder"], os.path.splitext(grouping["labview"])[0]+"time.txt")
    nikmeta_fpath = os.path.join(grouping["folder"], grouping["nikon_meta"])
    lfp_fpath =  os.path.join(grouping["folder"], grouping["lfp"])
    
    session = tps.TwoPhotonSession.init_and_process(nd2_fpath, nikmeta_fpath, lv_fpath, lvtime_fpath, lfp_fpath, matlab_2p_folder)
    
    bad_sessions[ind] = session

In [ ]:
bsl = ["9f57f50bf31a41c7af3da2a600bf7e42",
"171693d0988c458a96c8198c7b8cfc28",
"a6099849121f44ccbec237037971ab57",
"e40f26d410ab452e8f8d59e5394ae0fe",
"ae564f8c867f4f35aa971b6562c33a7c",
"ccc54ca6dff843cb8c9abcc1251adf74",
"65bff16a4cf04930a5cb14f489a8f99b",
"30dc55d1a5dc4b0286d132e72f208ca6",
"cdf06c160be947f59d6a03927d4e61e4",
"d7a5ac8e2bc74382b3db503a6a5a07a5",
"5ea6fd9c4cb542dbbc1f65305725cede",
"06ebcf354f5c41519669f187e16de364",
"50ab5da48817456fa63caa01a9f194ad",
"73a27053f4bf4ae1b4ad96064b6dabc0",
"cc7f02d61d66478b80f839221531c826",
"8dd54649e47046239ebafc56eeb8b5b2",
"44ca941252064dcabb0fe3d24a8dab49",
"21c83d0b69ec4585a9a11f4ce6c24b99",
"3dd896d33a0f42c698228fbe254ebd60",
"b4d9a66fb341473d8cd9d845a2a26155",
"a9694ce2973349cb9cb6b51f77c46b49",
"4083acdccf8c4f158aa0a5e188861c2b",
"db7a5cfc43954c19a439e71b9ff30c0f",
"99efe8d2c15940e49ee66018023d0e99",
"dc0f7ce9625940b5b2ca76d728458f0c",
"717a0fb5afa247b3915768acf410db70",
"757c430daa2349e198ddefa7a0277769",
"92062a977958443e83011619b34eabb8",
"3cb9934ddcc24cf7a922dca01bdb9448",
"1f6388ca8d7f48a2bd8dbe250749e413",
"7435d24e54b647e792ffd0b3c7bcda5a",
"b48b7bfa08b7424390c067b2695ff712",]


In [ ]:
for uuid in bsl:
    if uuid in uuids_lis:
        print(uuid)

In [ ]:
ex_ses = bad_sessions[11]

In [ ]:
ex_ses.time_offs_lfp_nik
# 11: 4.668004000000002
# 41: 16.23998
# 63: 4.544002000000001
# 76: 0

In [ ]:
#sessions_dict[uuids_lis[76]].shift_lfp(6.75)

In [ ]:
lfp_t, lfp_y = sessions_dict[uuids_lis[76]].lfp_movement()
lv_t = sessions_dict[uuids_lis[76]].belt_scn_dict["tsscn"]
lv_y = sessions_dict[uuids_lis[76]].belt_scn_dict["speed"]
lv_t_s = lv_t/1000.

In [ ]:
fig = plt.figure(figsize=(18, 10))
ampl = 100.

min_lfp = min(lfp_y[1000:])
max_lfp = max(lfp_y[1000:])

min_lv = min(lv_y)
max_lv = max(lv_y)

plt.plot(lv_t_s, ampl*(lv_y-min_lv)/(max_lv - min_lv), c="black")
plt.plot(lfp_t, ampl*(lfp_y-min_lfp)/(max_lfp - min_lfp) + ampl)
plt.gca()
plt.ylim((0, 300))
plt.xlim((700, 800))
plt.show()

In [ ]:
with h5py.File(export_fpath, "r+") as hf:
    for i in i_bad:
        uuid = uuids_lis[i]
        
        del hf[uuid]["lfp_mov_y"]
        del hf[uuid]["lfp_mov_t"]
        del hf[uuid]["lfp_t"] 
        del hf[uuid]["lfp_y"] 
        
        ses = bad_sessions[i]
        lfp_t, lfp_y = ses.lfp_lfp()
        mov_t, mov_y = ses.lfp_movement()
        hf.create_dataset(f"{uuid}/lfp_mov_t", data=np.array(mov_t))
        hf.create_dataset(f"{uuid}/lfp_mov_y", data=np.array(mov_y))
        
        hf.create_dataset(f"{uuid}/lfp_y", data=np.array(lfp_y))
        hf.create_dataset(f"{uuid}/lfp_t", data=np.array(lfp_t))
